In [1]:
import gymnasium as gym
import numpy as np

# On définit les ensembles S et A de sorte à pouvoir itérer dessus.
S = []
for current_sum in range(4,22):
    for dealers_one_card in range(1,11):
        S.append((current_sum,dealers_one_card, False))
        if current_sum > 11:
            S.append((current_sum,dealers_one_card, True))
A = [0,1]

env = gym.make('Blackjack-v1')

Le package gymnasium fournit des environnements (MDP) avec lesquel on peut interagir, et donc faire de l'apprentissage par renforcement.

On considère ici l'environnement Blackjack, qu'on peut décrire de la façon suivante. Un joueur joue contre un dealer. Le but du jeu est de faire en sorte que la somme de ses cartes soit la plus grande, mais sans dépasser 21. Les cartes J, Q, K valent 10. A peut valoir 1 ou 11, selon ce qui est le plus avantageux. 
- Au début du  jeu (d'un "épisode"), 2 cartes visibles sont distribuées au joueur, et 2 au dealer dont une est cachée.
- A chaque étape, le joueur choisit Hit (demander une carte supplémentaire) ou Stand (en rester là)
- Lorsque le joueur a terminé, le dealer se tire des cartes supplémentaires jusqu'à avoir au moins 17.

Consulter la documentation (https://gymnasium.farama.org/environments/toy_text/blackjack/) pour savoir comment le problème est modélisé.

Un environnement gymnasium fonctionne de la façon suivante.

In [2]:
# démarre un nouvel épisode. Un état est tiré au hasard.
s, _ = env.reset()
s

(15, 3, 0)

In [3]:
# si l'agent choisir l'action a=0, un nouvel état s est tiré, et un paiement r est obtenu

a = 0
s, r, terminated, truncated, _ = env.step(a)
print('Nouvel état', s)
print('Paiement', r)

# les variables terminated (resp. truncated) est un booléen qui est True lorsque l'épisode s'est achevé,
# ce qui revient à dire que le paiement ne pourra dorénavant être que nul
# (resp. lorsque l'épisode a été interrompu car le nombre d'étapes a atteint une limite fixée)

Nouvel état (15, 3, 0)
Paiement -1.0


**Question 1**: Proposer une politique simple sous la forme d'une fonction qui prend un état en entrée et qui renvoie une action (0 ou 1)

In [4]:
def pi(s):
    own_sum, dealer_visible_card, useable_ace = s
    if own_sum < min(dealer_visible_card*2,14):
        return 1
    else:
        return 0

**Question 2**: Pour la politique précédente, évaluer la quantité
$$\mathbb{E}_{\mu,\pi}\left[ \sum_{t=1}^{+\infty}R_t \right]$$
où $\mu$ est la distribution initiale d'états, en calculant la moyenne des paiements sur 100000 parties ("épisodes") (On considère donc ici $\gamma=1$).

In [5]:
cumul = 0
n_iter = 100000
for k in range(n_iter):
    s, _ = env.reset()
    
    terminated = False
    truncated = False

    while not (terminated or truncated):
        s, r, terminated, truncated, _ = env.step(pi(s))
        cumul += r
print('mean reward', cumul/n_iter)

mean reward -0.10688


**Question 3**: Définir une fonction qui prend en argument un état, une fonction action-valueur q (donnée sous la forme d'un dictionnaire dont les clés sont les états), ainsi qu'un epsilon, et qui avec probabilité epsilon renvoie une action tirée uniformément, et qui avec probabilité 1-epsilon renvoie une action choisie par une politique gloutonne par rapport à q.

In [6]:
def pi_eps_g(s,q,eps):
    if np.random.uniform() < eps or q[(s,0)] == q[(s,1)]:
        return np.random.randint(0,2)
    else:
        return 0 if q[(s,0)] > q[(s,1)] else 1

**Question 4**: Définir une fonction qui prend en argument une fonction action-valeur q et un epsilon, qui génère un épisode en utilisant la politique définie à la question précédente, et qui renvoie 3 listes, contenants les états, les actions et les paiements obtenus pendant l'épisode.

In [7]:
def generate_episode(q,eps):
    ss = []
    aa = []
    rr = []

    s, _ = env.reset()
    ss.append(s)
    
    terminated = False
    truncated = False

    while not (terminated or truncated):
        a = pi_eps_g(s,q,eps)
        aa.append(a)

        s, r, terminated, truncated, _ = env.step(a)

        ss.append(s)
        rr.append(r)

    return ss, aa, rr

**Question 5**: On cherche à mettre en oeuvre une itération de politique approchée. A chaque itération, la fonction état-valeur de la politique actuelle est estimée en générant un très grand nombre d'épisodes en utilisant non pas exactement la vraie politique gloutonne, mais celle correspond à la fonction `pi_esp_g` (pour une valeur de epsilon à choisir). Pour chaque paire (s,a) visitée, la somme des paiements des étapes futurs (de l'épisode concerné) est enregistrée, puis à la fin de l'itération, la composante correspondante de la fonction état-valeur est estimée par la moyenne des valeurs enregistrées. Expliciter la politique finalement obtenue.

In [8]:
def initialize_q_and_N():
    q = dict()
    N = dict() 
    for s in S:
        for a in A:
            q[(s,a)] = 1.
            N[(s,a)] = 0 # pour stocker le nombre de valeurs enregistrée pour chaque paire (s,a)
    return q, N

eps = .1
n_iter = 30
n_episodes_per_iter = 500000
previous_q, _ = initialize_q_and_N()
for iter in range(n_iter):
    print('iter', iter)
    current_cumul_q, current_N = initialize_q_and_N()
    for episode in range(n_episodes_per_iter):
        ss, aa, rr = generate_episode(previous_q,eps)
        for t in range(len(rr)):
            s = ss[t]
            a = aa[t]
            cumul_r = sum(rr[t:])
            current_cumul_q[(s,a)] = cumul_r+current_cumul_q[(s,a)]
            current_N[(s,a)] = 1 + current_N[(s,a)]

    if 0 in current_N.values():
        print('break')
        break

    previous_q = dict()
    for s in S:
        for a in A:
            previous_q[(s,a)] = current_cumul_q[(s,a)]/current_N[(s,a)]
            
# estimate average reward
cumul = 0
n_iter = 100000
for k in range(n_iter):
    s, _ = env.reset()
    
    terminated = False
    truncated = False

    while not (terminated or truncated):
        s, r, terminated, truncated, _ = env.step(pi_eps_g(s,previous_q,eps))
        cumul += r
print('mean reward', cumul/n_iter)


for s in S:
    if previous_q[(s,0)] >= previous_q[(s,1)]:
        print(s,'Stick')
    else:
        print(s,'Hit')

iter 0
iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9
iter 10
iter 11
iter 12
iter 13
iter 14
iter 15
iter 16
iter 17
iter 18
iter 19
iter 20
iter 21
iter 22
iter 23
iter 24
iter 25
iter 26
iter 27
iter 28
iter 29
mean reward -0.08755
(4, 1, False) Hit
(4, 2, False) Stick
(4, 3, False) Hit
(4, 4, False) Hit
(4, 5, False) Stick
(4, 6, False) Stick
(4, 7, False) Hit
(4, 8, False) Stick
(4, 9, False) Hit
(4, 10, False) Hit
(5, 1, False) Hit
(5, 2, False) Hit
(5, 3, False) Stick
(5, 4, False) Hit
(5, 5, False) Stick
(5, 6, False) Stick
(5, 7, False) Hit
(5, 8, False) Hit
(5, 9, False) Hit
(5, 10, False) Hit
(6, 1, False) Hit
(6, 2, False) Hit
(6, 3, False) Hit
(6, 4, False) Hit
(6, 5, False) Hit
(6, 6, False) Hit
(6, 7, False) Hit
(6, 8, False) Hit
(6, 9, False) Hit
(6, 10, False) Hit
(7, 1, False) Hit
(7, 2, False) Hit
(7, 3, False) Hit
(7, 4, False) Stick
(7, 5, False) Hit
(7, 6, False) Stick
(7, 7, False) Hit
(7, 8, False) Hit
(7, 9, False) Hit
(7, 10, False) Hit
(8, 1, 